In [ ]:
!pip install -q transformers torch pandas openpyxl

In [ ]:
import json
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from openai import OpenAI

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # Loads variables from .env into environment

In [ ]:
from openai import OpenAI

# Automatically read from environment (.env or system environment)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found. Please set it in your .env file.")

client = OpenAI(api_key=api_key)

In [ ]:
def llm_extract_points(comment: str) -> dict:
    """
    Calls GPT to split a comment into three lists:
    positive, negative, neutral.
    Output format MUST be strict JSON.
    """

    prompt = f"""
You are a system that extracts short, standalone bullet points from customer feedback.

For the input text, return JSON with this EXACT format:

{{
  "positive": ["point1", "point2", ...],
  "negative": ["point1", "point2", ...],
  "neutral":  ["point1", "point2", ...]
}}

Rules:
- Keep each point brief (max ~1 sentence).
- No summarising; preserve meaning.
- Put each point in the correct bucket.
- If a bucket is empty, return an empty list [].
- JSON only. No explanation.
- Do NOT add escape characters manually; return clean JSON.
--------------

INPUT:
"{comment}"
"""

    response = client.responses.create(
        model="gpt-4o-mini",   # can change to gpt-4.1 or gpt-4o
        input=prompt
    )

    # Extract model text
    text = response.output_text

    # Parse JSON
    try:
        data = json.loads(text)
    except:
        print("⚠️ JSON parse error, raw output:\n", text)
        raise

    # Guarantee the keys exist
    return {
        "positive": data.get("positive", []),
        "negative": data.get("negative", []),
        "neutral":  data.get("neutral", [])
    }

In [ ]:
CARDIFF_MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"

cardiff_tokenizer = AutoTokenizer.from_pretrained(CARDIFF_MODEL_NAME)
cardiff_model = AutoModelForSequenceClassification.from_pretrained(CARDIFF_MODEL_NAME)
cardiff_model.eval()

CARDIFF_ID2LABEL = {0: "Negative", 1: "Neutral", 2: "Positive"}

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def cardiff_predict_proba(text: str) -> np.ndarray:
    if not text or isinstance(text, float) and np.isnan(text):
        return np.array([1.0, 0.0, 0.0])
    encoded = cardiff_tokenizer(
        text, return_tensors="pt", truncation=True, max_length=256
    )
    with torch.no_grad():
        logits = cardiff_model(**encoded).logits
    probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
    return probs

In [ ]:
def argmax_label(pos: int, neg: int, neu: int) -> str:
    counts = {"Positive": pos, "Negative": neg, "Neutral": neu}
    return max(counts, key=counts.get)

def aggregate_cardiff(points: list[str]):
    """
    Returns:
      cardiff_counts: dict
      best_point: {point, probs, label} or None
    """
    counts = {"Positive": 0, "Negative": 0, "Neutral": 0}
    best = None
    best_score = -1

    for pt in points:
        probs = cardiff_predict_proba(pt)
        label_id = int(probs.argmax())
        label = CARDIFF_ID2LABEL[label_id]

        counts[label] += 1

        m = float(probs.max())
        if m > best_score:
            best_score = m
            best = {"point": pt, "probs": probs.tolist(), "label": label}

    return counts, best

In [ ]:
INPUT_FILE = "Sample Customer Feedback.xlsx"  # change if needed

df = pd.read_excel(INPUT_FILE)

print("Columns:", df.columns.tolist())

Columns: ['Submission ID', 'Comment']


In [ ]:
submission_col = "Submission"
comments_col = "Comment"

output_rows = []

for idx, row in df.iterrows():
    submission = row.get(submission_col)
    comment = str(row.get(comments_col) or "")

    # 1) GPT preprocessing
    points = llm_extract_points(comment)
    pos_pts = points["positive"]
    neg_pts = points["negative"]
    neu_pts = points["neutral"]

    # 2) LLM counts + overall
    pos_count = len(pos_pts)
    neg_count = len(neg_pts)
    neu_count = len(neu_pts)

    llm_overall = argmax_label(pos_count, neg_count, neu_count) if (pos_count + neg_count + neu_count) > 0 else "Neutral"

    # 3) Cardiff per-point aggregation
    all_pts = pos_pts + neg_pts + neu_pts
    cardiff_counts, best_point = aggregate_cardiff(all_pts)

    cardiff_pos = cardiff_counts["Positive"]
    cardiff_neg = cardiff_counts["Negative"]
    cardiff_neu = cardiff_counts["Neutral"]

    cardiff_overall = argmax_label(cardiff_pos, cardiff_neg, cardiff_neu) if (cardiff_pos + cardiff_neg + cardiff_neu) > 0 else "Neutral"

    cardiff_highest_json = json.dumps(best_point, ensure_ascii=False) if best_point else ""

    # 4) Build output row
    out = {
        "Submission": submission,
        "Comment": comment,
        "Positive Points": json.dumps(pos_pts, ensure_ascii=False),
        "Negative Points": json.dumps(neg_pts, ensure_ascii=False),
        "Neutral Points": json.dumps(neu_pts, ensure_ascii=False),
        "Positive Count": pos_count,
        "Negative Count": neg_count,
        "Neutral Count": neu_count,
        "LLM Overall Sentiment": llm_overall,
        "Cardiff Positive Count": cardiff_pos,
        "Cardiff Negative Count": cardiff_neg,
        "Cardiff Neutral Count": cardiff_neu,
        "Cardiff Overall Sentiment": cardiff_overall,
        "Cardiff Highest": cardiff_highest_json,
    }

    output_rows.append(out)

result_df = pd.DataFrame(output_rows)
result_df.head()

,Submission,Comment,Positive Points,Negative Points,Neutral Points,Positive Count,Negative Count,Neutral Count,LLM Overall Sentiment,Cardiff Positive Count,Cardiff Negative Count,Cardiff Neutral Count,Cardiff Overall Sentiment,Cardiff Highest
0,None,Give some gift to site team's during delivery...,[],[],"[""Give some gift to site teams during delivery...",0,0,1,Neutral,0,0,1,Neutral,"{""point"": ""Give some gift to site teams during..."
1,None,Delivering the material quickly can be improved.,[],"[""Delivering the material quickly can be impro...",[],0,1,0,Negative,1,0,0,Positive,"{""point"": ""Delivering the material quickly can..."
2,None,Need to improve QAQc,[],"[""Need to improve QAQc""]",[],0,1,0,Negative,0,0,1,Neutral,"{""point"": ""Need to improve QAQc"", ""probs"": [0...."
3,None,Share more infor about the product offer stand...,"[""Today order, tomorrow delivery.""]","[""Delivery item can be faster.""]","[""Share more info about the product offer stan...",1,1,3,Neutral,1,0,4,Neutral,"{""point"": ""Liaise with site directly."", ""probs..."
4,None,We are generally satisfied with Chong Cheong F...,"[""Generally satisfied with performance."", ""Dem...",[],[],4,0,0,Positive,3,0,1,Positive,"{""point"": ""Generally satisfied with performanc..."


In [ ]:
result_df.to_csv("sentiment_points_output.csv", index=False)
result_df.to_excel("sentiment_points_output.xlsx", index=False)

print("Saved output files!")

Saved output files!
